In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer, Metadata, Interpreter

In [2]:
def train_nlu(data, configuration, model_dir):
    training_data = load_data(data)
    trainer = Trainer(config.load(configuration))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir,fixed_model_name='faq_chatbot_nlu')

In [3]:
train_nlu_model_switch = True

In [25]:
if (train_nlu_model_switch):
    train_nlu('./data/data.json','spacy_configuration.json','./models/faq_chatbot_nlu')

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    1.8s finished


In [26]:
def run_nlu(query):
    interpreter = Interpreter.load('./models/faq_chatbot_nlu/default/faq_chatbot_nlu')
    return interpreter.parse(query)

In [28]:
query = 'what is the use of a tourist visa'

In [29]:
parsed_query = run_nlu(query)
parsed_query

C:\Users\Utkarsh Sah\Anaconda3\lib\site-packages\rasa_nlu\extractors\entity_synonyms.py:85: UserWarning: Failed to load synonyms file from './models/faq_chatbot_nlu/default/faq_chatbot_nlu\entity_synonyms.json'
  "".format(entity_synonyms_file))


{'entities': [{'confidence': 0.9928052551359788,
   'end': 33,
   'entity': 'subject',
   'extractor': 'ner_crf',
   'start': 21,
   'value': 'tourist visa'}],
 'intent': {'confidence': 0.58501234428080506, 'name': 'purpose'},
 'intent_ranking': [{'confidence': 0.58501234428080506, 'name': 'purpose'},
  {'confidence': 0.13018407585797612, 'name': 'definition'},
  {'confidence': 0.093871332735293023, 'name': 'procedure'},
  {'confidence': 0.071044706925857778, 'name': 'duration'},
  {'confidence': 0.02453255158114552, 'name': 'conditions'},
  {'confidence': 0.018571111102767872, 'name': 'address'},
  {'confidence': 0.013953292827593351, 'name': 'email'},
  {'confidence': 0.013373516541279025, 'name': 'contact'},
  {'confidence': 0.012752058141700589, 'name': 'validity'},
  {'confidence': 0.011885242540560519, 'name': 'benefits'}],
 'text': 'what is the use of a tourist visa'}